In [46]:
import os, re, random
from glob import glob
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from itertools import combinations 
from nltk.corpus import stopwords as stopwords_nltk 
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import re
from keybert import KeyBERT
from sklearn.utils import shuffle

In [84]:
mdsd_labeled_filepaths = glob('/media/dmlab/My Passport/DATA/BenchmarkDataset/MDSD/*_labeled_*.csv')
mdsd_unlabeled_filepaths = glob('/media/dmlab/My Passport/DATA/BenchmarkDataset/MDSD/*_unlabeled_*.csv')
save_dir = '/media/dmlab/My Passport/DATA/cross-domain/data'

* 4,000 unlabeled reviews per domain
* 2,000 labeled reviews per domain

In [47]:
lemmatizer = WordNetLemmatizer()
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)
removal_list = "|,‘, ’, ◇, ‘, ”,  ’, ·, \“, ·, △, ➤, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, ?, !,【,】, …, ◆,%"
stopwords = stopwords_nltk.words('english')
kw_model = KeyBERT()

def concat_dataframes(filepaths, sample_num=None):
    dfs = []
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        domain = os.path.basename(filepath).split('_')[0]
        df['domain'] = domain
        original_len = len(df)
        df.drop_duplicates(['text'], keep='last', inplace=True)   # 중복된 텍스트 제거
        print('[{}] Droped {} rows having duplicated text'.format(domain, original_len-len(df)))
        if sample_num is not None:
            df = df.sample(n=sample_num)   # 4000개 데이터만 랜덤하게 선택
        dfs.append(df)
    concat_df = pd.concat(dfs)
    concat_df = shuffle(concat_df)   # Shuffle
    concat_df.reset_index(inplace=True)   # Reset index
    return concat_df

def prepare_unlabeled_data(filepaths):
    def get_preprocessed_tokens(text):
        text = text.translate(str.maketrans(removal_list, ' '*len(removal_list)))   # 특수문자 제거
        text = re.sub(MULTIPLE_SPACES, ' ', text)   # 무의미한 공백 제거
        words = word_tokenize(text.lower())   # 소문자로 변경 후 tokenization
        nouns = [token for token, tag in pos_tag(words) if tag in ['NN', 'NNS', 'NNP', 'NNPS']]   # 명사 추출
        nouns = [lemmatizer.lemmatize(token) for token in nouns]   # lemmatization (e.g., movies -> movie)
        nouns = [token for token in nouns if token not in stopwords]   # 불용어 제거
        nouns = [token for token in nouns if len(token)>1]   # 길이가 1 이하인 단어 제거
        return nouns

    def mask_keywords(doc, keywords):
        words = doc.split()
        for i in range(len(words)):
            for keyword in keywords:
                if keyword.lower() in words[i].lower():
                    words[i] = '[UNK]'
        return ' '.join(words)
    
    def random_masked_text(num_of_unks, doc):
        words = doc.split()
        completed_unks = 0
        while True:
            random_idx = random.choice(range(len(words)))
            if completed_unks == num_of_unks: break
            if words[random_idx] != '[UNK]':
                words[random_idx] = '[UNK]'
                completed_unks += 1
        return ' '.join(words)

    concat_df = concat_dataframes(filepaths, sample_num=4000)

    # Keyword 추출 using KeyBERT
    concat_df['keywords'] = concat_df['text'].progress_apply(lambda x: \
        [word for (word, score) in kw_model.extract_keywords(x) if word in get_preprocessed_tokens(x)])
    
    # Keyword masking
    concat_df['masked_text'] = concat_df.progress_apply(lambda x: mask_keywords(x['text'], x['keywords']), axis=1)
    
    # Random word masking
    # keyword 개수 만큼 랜덤하게 단어를 골라서 [UNK]로 처리
    concat_df['random_masked_text'] = concat_df.progress_apply(lambda x: \
        random_masked_text(x['masked_text'].count('[UNK]'), x['text']), axis=1)
    return concat_df

unlabeled_df = prepare_unlabeled_data(mdsd_unlabeled_filepaths)
filepath = os.path.join(save_dir, 'MDSD_unlabeled.json')
unlabeled_df.to_json(filepath)
print('Created {}'.format(filepath))

labeled_df = concat_dataframes(mdsd_labeled_filepaths)
filepath = os.path.join(save_dir, 'MDSD_labeled.json')
labeled_df.to_json(filepath)
print('Created {}'.format(filepath))

[books] Droped 477734 rows having duplicated text
[dvd] Droped 39540 rows having duplicated text
[electronics] Droped 2213 rows having duplicated text
[kitchen] Droped 1218 rows having duplicated text


100%|██████████| 16000/16000 [00:01<00:00, 10075.06it/s]


Created /media/dmlab/My Passport/DATA/cross-domain/MDSD_unlabeled.json
[books] Droped 18 rows having duplicated text
[dvd] Droped 33 rows having duplicated text
[electronics] Droped 39 rows having duplicated text
[kitchen] Droped 23 rows having duplicated text
Created /media/dmlab/My Passport/DATA/cross-domain/MDSD_labeled.json


* Post-training용 데이터셋
    - 도메인 2개 (파일명에 알파벳 순으로 기재)
    - Post-training options
        1. Raw (baseline)
        2. keyword
        3. random word (baseline): keyword 개수 만큼 랜덤하게 단어를 골라서 [UNK]로 처리
    
> When generating the post-training data, each sentence in the target domain gets duplicated 10 times with different masks ~~and sentences pair~~.

In [85]:
def create_txt_for_post_training(docs, save_filepath, num_of_duplicates=10):    
    with open(save_filepath, 'w') as output_file:
        for _ in range(num_of_duplicates): # each sentence in the target domain gets duplicated 10 times
            for doc_idx, doc in enumerate(docs):
                output_file.write('{}\n\n'.format(doc))
        output_file.write('[EOD]')
    print(f'Created {save_filepath}')
    
domains = unlabeled_df.domain.unique()
for mode in ['raw', 'keyword', 'random']:
    for (domain1, domain2) in list(combinations(domains, 2)):
        df = unlabeled_df[unlabeled_df['domain'].isin([domain1, domain2])]
        if mode == 'raw':
            docs = df['text'].values
        elif mode == 'keyword':
            docs = df['masked_text'].values
        elif mode == 'random':
            docs = df['random_masked_text'].values

        save_filepath = os.path.join(save_dir, 'MDSD_{}_{}_for_post.txt'.format('&'.join(sorted([domain1, domain2])), mode))
        create_txt_for_post_training(docs, save_filepath)

Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_dvd&electronics_raw_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_electronics&kitchen_raw_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_books&electronics_raw_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_dvd&kitchen_raw_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_books&dvd_raw_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_books&kitchen_raw_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_dvd&electronics_keyword_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_electronics&kitchen_keyword_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_books&electronics_keyword_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-domain/data/MDSD_dvd&kitchen_keyword_for_post.txt
Created /media/dmlab/My Passport/DATA/cross-doma